---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [51]:
def blight_model():
    import pandas as pd
    import numpy as np
    import datetime
    import time
    from sklearn.model_selection import train_test_split
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import roc_curve, auc
    from sklearn.model_selection import GridSearchCV
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import roc_auc_score
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.preprocessing import LabelEncoder
    train = pd.read_csv("train.csv", encoding = 'ISO-8859-1', dtype={'zip_code': pd.np.str,'non_us_str_code': pd.np.str,'grafitti_status': pd.np.str})
    test = pd.read_csv("test.csv", encoding = 'ISO-8859-1', dtype={'zip_code': pd.np.str,'non_us_str_code': pd.np.str,'grafitti_status': pd.np.str})
    train = train[pd.notnull(train['compliance'])]
    train = train.astype({'compliance': int})
    train = train.fillna(0)
    test = test.fillna(0)
    address = pd.read_csv("addresses.csv")
    latlon = pd.read_csv("latlons.csv")
    temp = pd.merge(address, latlon, left_on="address", right_on="address", how="left")
    train = pd.merge(train, temp, left_on="ticket_id", right_on="ticket_id", how="left")
    test = pd.merge(test, temp, left_on="ticket_id", right_on="ticket_id", how="left")
    train['ticket_issued_date'] = pd.to_datetime(train['ticket_issued_date'])
    train['hearing_date'] = pd.to_datetime(train['hearing_date'])
    train['delay'] = (train['hearing_date'] - train['ticket_issued_date']).dt.days
    train['ticket_day_of_week'] = train['ticket_issued_date'].dt.dayofweek
    train['ticket_month'] = train['ticket_issued_date'].dt.month
    train['ticket_hour'] = train['ticket_issued_date'].dt.time
    t_12pm = datetime.datetime.strptime('12:00PM', '%I:%M%p').time()
    train['ticket_hour'] = np.where(train['ticket_hour'] > t_12pm, 'late', 'early')
    train['hearing_month'] = train['hearing_date'].dt.month
    train['hearing_day_of_week'] = train['hearing_date'].dt.dayofweek
    train['hearing_hour'] = train['hearing_date'].dt.time
    train['hearing_hour'] = np.where(train['hearing_hour'] > t_12pm, 'late', 'early')
    test['ticket_issued_date'] = pd.to_datetime(test['ticket_issued_date'])
    test['hearing_date'] = pd.to_datetime(test['hearing_date'])
    test['delay'] = (test['hearing_date'] - train['ticket_issued_date']).dt.days
    test['ticket_day_of_week'] = test['ticket_issued_date'].dt.dayofweek
    test['ticket_month'] = test['ticket_issued_date'].dt.month
    test['ticket_hour'] = test['ticket_issued_date'].dt.time
    test['ticket_hour'] = np.where(test['ticket_hour'] > t_12pm, 'late', 'early')
    test['hearing_month'] = test['hearing_date'].dt.month
    test['hearing_day_of_week'] = test['hearing_date'].dt.dayofweek
    test['hearing_hour'] = test['hearing_date'].dt.time
    test['hearing_hour'] = np.where(test['hearing_hour'] > t_12pm, 'late', 'early')
    train['MI'] = np.where(train['state']=='MI', 1, 0)
    test['MI'] = np.where(test['state']=='MI', 1, 0)
    train['violation_code'] = train['violation_code'].str.split('-').str[0]
    test['violation_code'] = test['violation_code'].str.split('-').str[0]
    train['vc'] = np.where(train['violation_code']=='9', 1, 0)
    test['vc'] = np.where(test['violation_code']=='9', 1, 0)
    rich_zip = ['48236', '48230', '48225', '48221', '48374', '48070', '48323', '48168', '48301', '48009', '48302', '48304', '48226', '48306', '48203', '48070', '48334', '48069', '48067', '48072', '48025', '48360', '48014', '48370', '48348', '49083', '48864', '48084', '48331', '48311', '48167', '48324', '48098', ]
    train['zip2'] = np.where(train['zip_code'].isin(rich_zip), 1, 0)
    test['zip2'] = np.where(test['zip_code'].isin(rich_zip), 1, 0)
    cats1 = train.loc[:, ['ticket_id', 'agency_name', 'hearing_month', 'disposition', 'ticket_hour', 'hearing_hour', 'state']]
    cats1['ticket_id'] = 'train' + cats1['ticket_id'].astype(str)
    cats2 = test.loc[:, ['ticket_id', 'agency_name', 'hearing_month', 'disposition', 'ticket_hour', 'hearing_hour', 'state']]
    cats2['ticket_id'] = 'test' + cats2['ticket_id'].astype(str)
    cats3 = cats1.append(cats2, ignore_index=True)
    # ucats_an = set(cats3['agency_name'])|{'<unknown>'}
    # cats3['agency_name']= pd.Categorical(cats3['agency_name'],categories=ucats_an).fillna('<unknown>').codes
    # ucats_hm = set(cats3['hearing_month'])|{'<unknown>'}
    # cats3['hearing_month']= pd.Categorical(cats3['hearing_month'],categories=ucats_hm).fillna('<unknown>').codes
    # ucats_mi = set(cats3['MI'])|{'<unknown>'}
    # cats3['MI']= pd.Categorical(cats3['MI'],categories=ucats_mi).fillna('<unknown>').codes
    # ucats_zip2 = set(cats3['zip2'])|{'<unknown>'}
    # cats3['zip2']= pd.Categorical(cats3['zip2'],categories=ucats_zip2).fillna('<unknown>').codes
    # ucats_disp = set(cats3['disposition'])|{'<unknown>'}
    # cats3['disposition']= pd.Categorical(cats3['disposition'],categories=ucats_disp).fillna('<unknown>').codes
    cats3 = pd.get_dummies(cats3, columns=['agency_name', 'hearing_month', 'disposition', 'ticket_hour', 'hearing_hour'], prefix = ['a', 'b', 'c', 'd', 'e'])
    cats4 = cats3[cats3['ticket_id'].str.contains('train')]
    cats5 = cats3[cats3['ticket_id'].str.contains('test')]
    cats4['ticket_id'] = cats4['ticket_id'].str.split('ain').str[1]
    cats4['ticket_id'] = pd.to_numeric(cats4['ticket_id'])
    cats5['ticket_id'] = cats5['ticket_id'].str.split('est').str[1]
    cats5['ticket_id'] = pd.to_numeric(cats5['ticket_id'])
    train = pd.merge(train, cats4, left_on="ticket_id", right_on="ticket_id", how="left")
    test = pd.merge(test, cats5, left_on="ticket_id", right_on="ticket_id", how="left")
    # features = 
    X = train.loc[:, ['late_fee', 'fine_amount', 'discount_amount', 'judgment_amount', 'lat', 'lon', 'delay', 'c_Responsible (Fine Waived) by Admis', 'c_Responsible (Fine Waived) by Deter', 'c_Responsible - Compl/Adj by Default', 'c_Responsible - Compl/Adj by Determi', 'c_Responsible by Admission', 'c_Responsible by Default', 'c_Responsible by Determination', 'c_Responsible by Dismissal']]
    X1 = test.loc[:, ['late_fee', 'fine_amount', 'discount_amount', 'judgment_amount', 'lat', 'lon', 'delay', 'c_Responsible (Fine Waived) by Admis', 'c_Responsible (Fine Waived) by Deter', 'c_Responsible - Compl/Adj by Default', 'c_Responsible - Compl/Adj by Determi', 'c_Responsible by Admission', 'c_Responsible by Default', 'c_Responsible by Determination', 'c_Responsible by Dismissal']]
    y = train.iloc[:, 33]
    X = X.fillna(0)
    X1 = X1.fillna(0)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    #clf = DecisionTreeClassifier(max_depth=5).fit(X_train, y_train)
    gbc = GradientBoostingClassifier().fit(X_train, y_train)
    # # #rfc = RandomForestClassifier(max_depth=3).fit(X_train, y_train)
    # # # #lr = LogisticRegression().fit(X_train, y_train)
    # # #scaler = MinMaxScaler()
    # # #X_train_scaled = scaler.fit_transform(X_train)
    # # #X_test_scaled = scaler.transform(X_test)
    #score1 = gbc.score(X_train, y_train)
    #score2 = gbc.score(X_test, y_test)
    #print((score1, score2))
    #important_features = pd.Series(data=gbc.feature_importances_,index=X.columns)
    #important_features.sort_values(ascending=False,inplace=True)
    #print(important_features)
    #y_score_gbc = gbc.decision_function(X_test)
    #fpr_gbc, tpr_gbc, _ = roc_curve(y_test, y_score_gbc)
    #roc_auc_gbc = auc(fpr_gbc, tpr_gbc)
    #print(roc_auc_gbc)
    yprob = gbc.predict_proba(X1)
    answer = yprob[:,1]
    answer1 = pd.Series(answer, index=test['ticket_id'])
    return answer1

In [2]:
# import pandas as pd
# import numpy as np
# import datetime
# import time
# from sklearn.model_selection import train_test_split
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import roc_curve, auc
# from sklearn.model_selection import GridSearchCV
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import roc_auc_score
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import LabelEncoder
# train = pd.read_csv("train.csv", encoding = 'ISO-8859-1', dtype={'zip_code': pd.np.str,'non_us_str_code': pd.np.str,'grafitti_status': pd.np.str})
# test = pd.read_csv("test.csv", encoding = 'ISO-8859-1', dtype={'zip_code': pd.np.str,'non_us_str_code': pd.np.str,'grafitti_status': pd.np.str})
# train = train[pd.notnull(train['compliance'])]
# train = train.astype({'compliance': int})
# train = train.fillna(0)
# test = test.fillna(0)
# address = pd.read_csv("addresses.csv")
# latlon = pd.read_csv("latlons.csv")
# temp = pd.merge(address, latlon, left_on="address", right_on="address", how="left")
# train = pd.merge(train, temp, left_on="ticket_id", right_on="ticket_id", how="left")
# test = pd.merge(test, temp, left_on="ticket_id", right_on="ticket_id", how="left")
# train['ticket_issued_date'] = pd.to_datetime(train['ticket_issued_date'])
# train['hearing_date'] = pd.to_datetime(train['hearing_date'])
# train['delay'] = (train['hearing_date'] - train['ticket_issued_date']).dt.days
# train['ticket_day_of_week'] = train['ticket_issued_date'].dt.dayofweek
# train['ticket_month'] = train['ticket_issued_date'].dt.month
# train['ticket_hour'] = train['ticket_issued_date'].dt.time
# t_12pm = datetime.datetime.strptime('12:00PM', '%I:%M%p').time()
# train['ticket_hour'] = np.where(train['ticket_hour'] > t_12pm, 'late', 'early')
# train['hearing_month'] = train['hearing_date'].dt.month
# train['hearing_day_of_week'] = train['hearing_date'].dt.dayofweek
# train['hearing_hour'] = train['hearing_date'].dt.time
# train['hearing_hour'] = np.where(train['hearing_hour'] > t_12pm, 'late', 'early')
# test['ticket_issued_date'] = pd.to_datetime(test['ticket_issued_date'])
# test['hearing_date'] = pd.to_datetime(test['hearing_date'])
# test['delay'] = (test['hearing_date'] - train['ticket_issued_date']).dt.days
# test['ticket_day_of_week'] = test['ticket_issued_date'].dt.dayofweek
# test['ticket_month'] = test['ticket_issued_date'].dt.month
# test['ticket_hour'] = test['ticket_issued_date'].dt.time
# test['ticket_hour'] = np.where(test['ticket_hour'] > t_12pm, 'late', 'early')
# test['hearing_month'] = test['hearing_date'].dt.month
# test['hearing_day_of_week'] = test['hearing_date'].dt.dayofweek
# test['hearing_hour'] = test['hearing_date'].dt.time
# test['hearing_hour'] = np.where(test['hearing_hour'] > t_12pm, 'late', 'early')
# train['MI'] = np.where(train['state']=='MI', 1, 0)
# test['MI'] = np.where(test['state']=='MI', 1, 0)
# train['violation_code'] = train['violation_code'].str.split('-').str[0]
# test['violation_code'] = test['violation_code'].str.split('-').str[0]
# train['vc'] = np.where(train['violation_code']=='9', 1, 0)
# test['vc'] = np.where(test['violation_code']=='9', 1, 0)
# rich_zip = ['48236', '48230', '48225', '48221', '48374', '48070', '48323', '48168', '48301', '48009', '48302', '48304', '48226', '48306', '48203', '48070', '48334', '48069', '48067', '48072', '48025', '48360', '48014', '48370', '48348', '49083', '48864', '48084', '48331', '48311', '48167', '48324', '48098', ]
# train['zip2'] = np.where(train['zip_code'].isin(rich_zip), 1, 0)
# test['zip2'] = np.where(test['zip_code'].isin(rich_zip), 1, 0)
# cats1 = train.loc[:, ['ticket_id', 'agency_name', 'hearing_month', 'disposition', 'ticket_hour', 'hearing_hour', 'state']]
# cats1['ticket_id'] = 'train' + cats1['ticket_id'].astype(str)
# cats2 = test.loc[:, ['ticket_id', 'agency_name', 'hearing_month', 'disposition', 'ticket_hour', 'hearing_hour', 'state']]
# cats2['ticket_id'] = 'test' + cats2['ticket_id'].astype(str)
# cats3 = cats1.append(cats2, ignore_index=True)
# # ucats_an = set(cats3['agency_name'])|{'<unknown>'}
# # cats3['agency_name']= pd.Categorical(cats3['agency_name'],categories=ucats_an).fillna('<unknown>').codes
# # ucats_hm = set(cats3['hearing_month'])|{'<unknown>'}
# # cats3['hearing_month']= pd.Categorical(cats3['hearing_month'],categories=ucats_hm).fillna('<unknown>').codes
# # ucats_mi = set(cats3['MI'])|{'<unknown>'}
# # cats3['MI']= pd.Categorical(cats3['MI'],categories=ucats_mi).fillna('<unknown>').codes
# # ucats_zip2 = set(cats3['zip2'])|{'<unknown>'}
# # cats3['zip2']= pd.Categorical(cats3['zip2'],categories=ucats_zip2).fillna('<unknown>').codes
# # ucats_disp = set(cats3['disposition'])|{'<unknown>'}
# # cats3['disposition']= pd.Categorical(cats3['disposition'],categories=ucats_disp).fillna('<unknown>').codes
# cats3 = pd.get_dummies(cats3, columns=['agency_name', 'hearing_month', 'disposition', 'ticket_hour', 'hearing_hour'], prefix = ['a', 'b', 'c', 'd', 'e'])
# cats4 = cats3[cats3['ticket_id'].str.contains('train')]
# cats5 = cats3[cats3['ticket_id'].str.contains('test')]
# cats4['ticket_id'] = cats4['ticket_id'].str.split('ain').str[1]
# cats4['ticket_id'] = pd.to_numeric(cats4['ticket_id'])
# cats5['ticket_id'] = cats5['ticket_id'].str.split('est').str[1]
# cats5['ticket_id'] = pd.to_numeric(cats5['ticket_id'])
# train = pd.merge(train, cats4, left_on="ticket_id", right_on="ticket_id", how="left")
# test = pd.merge(test, cats5, left_on="ticket_id", right_on="ticket_id", how="left")
# # features = 
# X = train.loc[:, ['ticket_id', 'late_fee', 'fine_amount', 'discount_amount', 'judgment_amount', 'lat', 'lon', 'delay', 'c_Responsible (Fine Waived) by Admis', 'c_Responsible (Fine Waived) by Deter', 'c_Responsible - Compl/Adj by Default', 'c_Responsible - Compl/Adj by Determi', 'c_Responsible by Admission', 'c_Responsible by Default', 'c_Responsible by Determination', 'c_Responsible by Dismissal']]
# X1 = test.loc[:, ['ticket_id', 'late_fee', 'fine_amount', 'discount_amount', 'judgment_amount', 'lat', 'lon', 'delay', 'c_Responsible (Fine Waived) by Admis', 'c_Responsible (Fine Waived) by Deter', 'c_Responsible - Compl/Adj by Default', 'c_Responsible - Compl/Adj by Determi', 'c_Responsible by Admission', 'c_Responsible by Default', 'c_Responsible by Determination', 'c_Responsible by Dismissal']]
# y = train.iloc[:, 33]
# X = X.fillna(0)
# X1 = X1.fillna(0)
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
# #clf = DecisionTreeClassifier(max_depth=5).fit(X_train, y_train)
# gbc = GradientBoostingClassifier().fit(X_train, y_train)
# # # #rfc = RandomForestClassifier(max_depth=3).fit(X_train, y_train)
# # # # #lr = LogisticRegression().fit(X_train, y_train)
# # # #scaler = MinMaxScaler()
# # # #X_train_scaled = scaler.fit_transform(X_train)
# # # #X_test_scaled = scaler.transform(X_test)
# #score1 = gbc.score(X_train, y_train)
# #score2 = gbc.score(X_test, y_test)
# #print((score1, score2))
# #important_features = pd.Series(data=gbc.feature_importances_,index=X.columns)
# #important_features.sort_values(ascending=False,inplace=True)
# #print(important_features)
# #y_score_gbc = gbc.decision_function(X_test)
# #fpr_gbc, tpr_gbc, _ = roc_curve(y_test, y_score_gbc)
# #roc_auc_gbc = auc(fpr_gbc, tpr_gbc)
# #print(roc_auc_gbc)
# yprob = gbc.predict_proba(X1)
# answer = yprob[:,1]
# answer1 = pd.Series(answer, index=test['ticket_id'])
# answer1

In [6]:
# test.columns

In [7]:
# train.state_x.value_counts()

In [ ]:
# test_data.columns

In [ ]:
# test_data.state_fee.value_counts()